# SpringBoard Capstone3: Coronavirus Tweets Sentiment Analysis

    Data from Kaggle.com: https://www.kaggle.com/datatattle/covid-19-nlp-text-classification/metadata
## XGboost

In [7]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
# from wordcloud import WordCloud
import re
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [12]:
tweet = pd.read_csv('ProcessedTweets', index_col = 0)
tweet.fillna('', inplace=True)
tweet.head()
tweet.isna().any()

test = pd.read_csv('ProcessedTweets', index_col = 0)
test.fillna('', inplace=True)
test.head()
test.isna().any()

text         False
Sentiment    False
dtype: bool

In [16]:
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.model_selection import train_test_split 

X = tweet['text'] 
y = tweet['Sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 42)

Count_vect = CountVectorizer(ngram_range=(1, 1), max_df=0.7, min_df=15)

Count_train = Count_vect.fit_transform(X_train)
Count_test = Count_vect.transform(X_test)

# Print the features
print('# of feature names', len(Count_vect.get_feature_names()))
print(Count_vect.get_feature_names()[100:200])

# of feature names 4507
['alcoholbased', 'aldi', 'alegedly', 'alergies', 'alert', 'alive', 'almost', 'alone', 'along', 'alongside', 'alow', 'alowed', 'alowing', 'alows', 'already', 'also', 'alternative', 'alternatives', 'although', 'always', 'amazing', 'amazon', 'ambulance', 'america', 'american', 'americans', 'amid', 'amidst', 'amo', 'among', 'amongst', 'amount', 'amounts', 'amp', 'ample', 'analysis', 'analyst', 'analysts', 'analytics', 'andor', 'angela', 'angeles', 'angry', 'animal', 'animals', 'another', 'anounce', 'anounced', 'anouncement', 'anouncements', 'anounces', 'anouncing', 'answer', 'answered', 'answers', 'anti', 'antibacterial', 'anticipate', 'anual', 'anxiety', 'anxious', 'anybody', 'anymore', 'anyone', 'anything', 'anyway', 'anywhere', 'ap', 'aparel', 'aparently', 'apart', 'apartment', 'apeal', 'apear', 'apears', 'aple', 'aplications', 'aplied', 'aplies', 'aply', 'apocalypse', 'apr', 'apreciate', 'apreciated', 'apreciation', 'april', 'aproach', 'apropriate', 'aprove', 'a

In [41]:
pip install xgboost

     |████████████████████████████████| 1.3 MB 2.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_XG = label_encoder.fit_transform(y_train)
print(y_XG[:5])
y_test_XG = label_encoder.fit_transform(y_test)
print(y_test_XG[:5])

[1 4 3 4 0]
[3 0 4 4 2]


In [8]:
from xgboost import XGBClassifier

XGmodel = XGBClassifier(use_label_encoder=False)
XGmodel.fit(Count_train, y_XG)

# make predictions for test data

y_pred_XG = XGmodel.predict(Count_test)

# evaluate predictions
print(classification_report(y_test_XG, y_pred_XG))
accuracy = accuracy_score(y_test_XG, y_pred_XG)
CM_XG = confusion_matrix(y_test_XG, y_pred_XG)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print(CM_XG)

[22:33:54] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       0.63      0.50      0.56       790
           1       0.72      0.55      0.62      1005
           2       0.56      0.47      0.51      1516
           3       0.55      0.84      0.67      1176
           4       0.55      0.57      0.56      1687

    accuracy                           0.58      6174
   macro avg       0.60      0.59      0.58      6174
weighted avg       0.59      0.58      0.58      6174

Accuracy: 58.36%
[[395  12 250  81  52]
 [  6 550  33  67 349]
 [164  41 711 322 278]
 [ 11  11  66 989  99]
 [ 47 149 206 327 958]]


In [10]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [20, 200],
    #'colsample_bytree': [0.2, 0.8],
    'max_depth': [10,50],
    #'reg_alpha': [1.1, 1.3],
    #'reg_lambda': [1.1, 1.3],
    #'subsample': [0.5, 0.9]
}

grid_GX = GridSearchCV(XGBClassifier(use_label_encoder=False), param_grid=param_grid, cv = 5, scoring='accuracy', verbose=10)
GX_grid_results = grid_GX.fit(Count_train,y_XG)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 1/5; 1/4] START max_depth=10, n_estimators=20...............................
[22:36:05] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 1/5; 1/4] END .............max_depth=10, n_estimators=20; total time=   7.6s
[CV 2/5; 1/4] START max_depth=10, n_estimators=20...............................
[22:36:12] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 2/5; 1/4] END .............max_depth=10, n_estimators=20; total time=   7.7s
[CV 3/5; 1/4] START max_depth=10, n_estimators=20.

[CV 4/5; 4/4] END ............max_depth=50, n_estimators=200; total time= 4.3min
[CV 5/5; 4/4] START max_depth=50, n_estimators=200..............................
[23:04:50] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 5/5; 4/4] END ............max_depth=50, n_estimators=200; total time= 4.3min
[23:09:06] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [11]:
result=pd.DataFrame(grid_GX.cv_results_)
result

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,7.570562,0.087050,0.038165,0.003925,10,20,"{'max_depth': 10, 'n_estimators': 20}",0.500500,0.512220,0.518222,0.519297,0.524728,0.514993,0.008264,4
1,73.185942,3.145271,0.768813,0.285830,10,200,"{'max_depth': 10, 'n_estimators': 200}",0.602687,0.607403,0.614120,0.619497,0.616924,0.612126,0.006209,1
2,38.379387,0.124658,0.170928,0.002723,50,20,"{'max_depth': 50, 'n_estimators': 20}",0.554095,0.557096,0.560955,0.568897,0.566038,0.561416,0.005474,3
3,274.072275,34.168977,2.022613,0.312483,50,200,"{'max_depth': 50, 'n_estimators': 200}",0.597256,0.593969,0.604402,0.602773,0.607776,0.601235,0.004976,2


In [17]:
XGmodel = XGBClassifier(use_label_encoder=False, max_depth = 10, n_estimators = 200)
XGmodel.fit(Count_train, y_XG)

# make predictions for test data

y_pred_XG = XGmodel.predict(Count_test)

# evaluate predictions
print(classification_report(y_test_XG, y_pred_XG))
accuracy = accuracy_score(y_test_XG, y_pred_XG)
CM_XG = confusion_matrix(y_test_XG, y_pred_XG)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print(CM_XG)

[03:06:15] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
              precision    recall  f1-score   support

           0       0.66      0.55      0.60       790
           1       0.72      0.59      0.65      1005
           2       0.59      0.54      0.57      1516
           3       0.65      0.85      0.74      1176
           4       0.58      0.61      0.60      1687

    accuracy                           0.63      6174
   macro avg       0.64      0.63      0.63      6174
weighted avg       0.63      0.63      0.63      6174

Accuracy: 62.96%
[[ 436    9  256   46   43]
 [   8  592   27   29  349]
 [ 167   43  822  218  266]
 [  15    9   72 1000   80]
 [  36  166  209  239 1037]]


## Did not do more steps, the XGboost is similar as SVC using tfidf